In [12]:
import re

import numpy as np
import keras
from keras.layers import Dense, Dropout, LSTM
from collections import defaultdict

In [13]:
text = open('shakespeare.txt', 'r', encoding='utf-8').read()
text = text.lower()
text = re.sub(r'[^a-zа-я\s.]+', '', text)
text = re.sub(r'\s+', ' ', text)
splited_text = text.split('.')[:100]
splited_text

['first citizen before we proceed any further hear me speak',
 ' all speak speak',
 ' first citizen you are all resolved rather to die than to famish all resolved',
 ' resolved',
 ' first citizen first you know caius marcius is chief enemy to the people',
 ' all we knowt we knowt',
 ' first citizen let us kill him and well have corn at our own price',
 ' ist a verdict all no more talking ont let it be done away away second citizen one word good citizens',
 ' first citizen we are accounted poor citizens the patricians good',
 ' what authority surfeits on would relieve us if they would yield us but the superfluity while it were wholesome we might guess they relieved us humanely but they think we are too dear the leanness that afflicts us the object of our misery is as an inventory to particularise their abundance our sufferance is a gain to them let us revenge this with our pikes ere we become rakes for the gods know i speak this in hunger for bread not in thirst for revenge',
 ' second 

In [14]:
chars = sorted(list(set(text)))
char2int = dict()
int2char = dict()

for i, c in enumerate(chars):
    char2int[c] = i
    int2char[i] = c

In [15]:
window_length = 25
dataX = []
dataY = []
for sentence in splited_text:
  sentence += '.'
  for i in range(0, len(sentence) - window_length):
    dataX.append(sentence[i:i + window_length])
    dataY.append(sentence[i + window_length])

In [16]:
n_sentences = len(dataX)
X = np.zeros((n_sentences, window_length, len(chars)), dtype=np.bool_)
y = np.zeros((n_sentences, len(chars)), dtype=np.bool_)
for i, sentence in enumerate(dataX):
    for j, char in enumerate(sentence):
        X[i, j, char2int[char]] = True
    y[i, char2int[dataY[i]]] = True

In [17]:
model = keras.Sequential(
    [
        LSTM(256, input_shape=X.shape[1:], return_sequences=True),
        Dropout(0.2),
        LSTM(256),
        Dense(y.shape[1], activation='softmax')
    ]
)
model.compile(loss="categorical_crossentropy", optimizer='adam')
model.fit(X, y, batch_size=128, epochs=40)

Epoch 1/40
89/89 [==============================] - 26s 254ms/step - loss: 2.9100
Epoch 2/40
89/89 [==============================] - 22s 252ms/step - loss: 2.7993
Epoch 3/40
89/89 [==============================] - 23s 256ms/step - loss: 2.4946
Epoch 4/40
89/89 [==============================] - 23s 253ms/step - loss: 2.3009
Epoch 5/40
89/89 [==============================] - 22s 251ms/step - loss: 2.2062
Epoch 6/40
89/89 [==============================] - 23s 254ms/step - loss: 2.1252
Epoch 7/40
89/89 [==============================] - 23s 255ms/step - loss: 2.0466
Epoch 8/40
89/89 [==============================] - 23s 263ms/step - loss: 1.9758
Epoch 9/40
89/89 [==============================] - 23s 253ms/step - loss: 1.9121
Epoch 10/40
89/89 [==============================] - 23s 253ms/step - loss: 1.8470
Epoch 11/40
89/89 [==============================] - 23s 254ms/step - loss: 1.7874
Epoch 12/40
89/89 [==============================] - 23s 256ms/step - loss: 1.7301
Epoch 13/40
8

In [29]:
def generate(pattern):
    print('Pattern = "' + pattern + '"')
    for _ in range(100):
        x = np.zeros((1, window_length, len(chars)))
        for j, char in enumerate(pattern):
            x[0, j, char2int[char]] = 1.0
        preds = model.predict(x)
        next_char = int2char[np.argmax(preds)]
        pattern = pattern[1:] + next_char
        print(str(next_char), end="")

In [22]:
pattern = "first senator"

generate(pattern)

Pattern = "first senator"
b cncbcc c icwwwwyeeeebehessiassoosrdssssg ddv            wwwdmcpcc        ww wwdmcmcfpcmpc c ccyf  

In [27]:
index = np.random.randint(0, n_sentences - 1)
pattern = dataX[index]

generate(pattern)

Pattern = "he did it to please his m"
other and to be partly proud which he is even till the altitude of his virtue.d to give hig good for

In [28]:
pattern = "be or not to be thats the"

generate(pattern)

Pattern = "be or not to be thats the"
 seat o the brain and through the cranks and offices of man the strongest nerves and small inferior 

In [33]:
markov_window_length = 15
markov_dataX = []
markov_dataY = []
for sentence in text.split('.'):
  sentence += '.'
  for i in range(0, len(sentence) - markov_window_length):
    markov_dataX.append(sentence[i:i + markov_window_length])
    markov_dataY.append(sentence[i + markov_window_length])

nodes = defaultdict(lambda: defaultdict(lambda: 0))
for sentence, symbol in zip(markov_dataX, markov_dataY):
  nodes[sentence][symbol] += 1

In [38]:
index = np.random.randint(0, len(markov_dataX) - 1)

pattern = markov_dataX[index]
print('Pattern = "' + pattern + '"')
for i in range(100):
    next_chars_pool = [symbol for symbol in nodes[pattern]]
    preds = np.array([w for w in nodes[pattern].values()])
    preds = preds / preds.sum()
    if len(preds) == 0:
      break
    next_char = next_chars_pool[np.argmax(preds)]
    pattern = pattern[1:] + next_char
    print(str(next_char), end='')

Pattern = "omely swain to "
sit upon a hill as i do now to carve out dials quaintly point by point thereby to see the minutes ho